In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from skimage.transform import rotate

## Get files from a path

In [ ]:
def get_files(path):
    files = list()
    
    # /kaggle/input/dataset/train
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    
    return files

In [ ]:
train_files_path = get_files('/kaggle/input/dataset/train')

train_csv = pd.read_csv('/kaggle/input/dataset/train.csv')

print("Train files : {0}".format(len(train_files_path)))

In [ ]:
train_dict = dict(zip(train_csv.Image, train_csv.target))
print(train_dict['96.jpg'])

### Split into training and validation set

In [ ]:
split = int(len(train_files_path) * 0.83)
random.shuffle(train_files_path)
train_files = train_files_path[:split]
val_files = train_files_path[split:]

print("Training files {0}".format(len(train_files)))
print("Validation files {0}".format(len(val_files)))

### Converting dance forms into numbers

In [ ]:
unique_labels = list(set(train_dict.values()))
unique_labels_mapping = dict()
for index in range(len(unique_labels)):
    unique_labels_mapping[unique_labels[index]] = index

total_classes = len(unique_labels)
print("Number of classes : {0}".format(total_classes))

In [ ]:
def get_one_hot_encoded_mask(value, num_labels):
    return to_categorical(value, num_classes = num_labels)

### Setting up hyper-parameters

In [ ]:
img_width, img_height = 256, 256
batch_size = 16
epochs = 10


### Creating Augmentations

In [ ]:
def rotate_image(image, angle_list):
    rotated_images = list()
    for angle in angle_list:
        rotatef_images.append(rotate(image,angle))
    
    return rotated_images

In [ ]:
def scaleDown_image(image, fx=0.6, fy=0.6):
    return cv2.resize(img, None, fx= 0.6, fy= 0.6, interpolation= cv2.INTER_LINEAR)

In [ ]:
def scaleUp_image(image, fx = 2, fy = 2):
    return cv2.resize(img, None, fx = fx, fy = fy, interpolation= cv2.INTER_LINEAR)

In [ ]:
def horizontal_flip(img):
    return img[: , ::-1]

## Creating DataGenerator

In [ ]:
class DataGenerator:
    def __init__(self, train_files, valid_files, labels_dict, batch_size = 16):
        self.train_files = train_files
        self.valid_files = valid_files
        self.labels_dict = labels_dict
        self.batch_size = batch_size
        
    def train_generator(self):
        num_images = len(self.train_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.train_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.train_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name)]]
                    y_batch.append(label)
                    
                    rotated_images = rotate_image(img, [45, 60, -45, -60])
                    for rotated_image in rotated_images:
                        x_batch.append(rotate_image)
                        y_batch.append(label)
                    
                    x_batch.append(scaleDown_image(img))
                    y_batch.append(label)
                    
                    x_batch.append(scaleUp_image(img))
                    y_batch.append(label)
                    
                    x_batch.append(horizontal_flip(img))
                    y_batch.append(label)
                
                yield (np.array(x_batch), np.array(y_batch))
    
    def valid_generator(self):
        num_images = len(self.valid_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.valid_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.valid_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name)]]
                    y_batch.append(label)
                    
                    rotated_images = rotate_image(img, [45, 60, -45, -60])
                    for rotated_image in rotated_images:
                        x_batch.append(rotate_image)
                        y_batch.append(label)
                    
                    x_batch.append(scaleDown_image(img))
                    y_batch.append(label)
                    
                    x_batch.append(scaleUp_image(img))
                    y_batch.append(label)
                    
                    x_batch.append(horizontal_flip(img))
                    y_batch.append(label)
                
                yield (np.array(x_batch), np.array(y_batch))